In [92]:
import numpy as np
import pprint
import json

Step 1: Calculate expected side products (without affecting benzoate part)

In [93]:
Ol2Et = {
    'st_ms' : 184.15,
    'updt_ms' : 168.324,
    'rxn' : 'smOH->OC12H25_',
    'max_rxn' : 3,
    'msg' : '* OH group(s) on trioxyl methyl-benzoate substituted by dodecyl group(s). '
}

In [94]:
all_ms_Ol2Et = {
    'rxn_n' : [],
    'ms' : [],
    'msg' : [],
    'name' : []
}

def all_ms_updt(all_ms_dict, new_rxn_n, new_ms, new_msg, rxn_type, pre_rxn = ''):
    all_ms_dict['rxn_n'].append(new_rxn_n)
    all_ms_dict['ms'].append(new_ms)
    all_ms_dict['msg'].append(new_msg)
    new_name = pre_rxn + rxn_type + str(new_rxn_n)
    all_ms_dict['name'].append(new_name)
    # print(all_ms_dict)
    return all_ms_dict

# calculate all the possible combinations with OH substitution

for i in range(Ol2Et['max_rxn'] + 1):
    new_ms = Ol2Et['st_ms'] + Ol2Et['updt_ms'] * i
    # print(new_ms)
    new_msg = str(i) + ' ' + Ol2Et['msg']
    # print(new_msg)
    rxn_type = Ol2Et['rxn']
    all_ms_updt(all_ms_Ol2Et, i, new_ms, new_msg, rxn_type, pre_rxn = '')

pprint.pprint(all_ms_Ol2Et)
    


{'ms': [184.15, 352.47400000000005, 520.798, 689.1220000000001],
 'msg': ['0 * OH group(s) on trioxyl methyl-benzoate substituted by dodecyl '
         'group(s). ',
         '1 * OH group(s) on trioxyl methyl-benzoate substituted by dodecyl '
         'group(s). ',
         '2 * OH group(s) on trioxyl methyl-benzoate substituted by dodecyl '
         'group(s). ',
         '3 * OH group(s) on trioxyl methyl-benzoate substituted by dodecyl '
         'group(s). '],
 'name': ['smOH->OC12H25_0',
          'smOH->OC12H25_1',
          'smOH->OC12H25_2',
          'smOH->OC12H25_3'],
 'rxn_n': [0, 1, 2, 3]}


Step 2: Work out potential products generated by ester hydrolysis from the previous structures.

In [95]:
Est2Acd = {
    'st_ms_lst' : all_ms_Ol2Et['ms'],
    'updt_ms' : -14.027,
    'rxn' : '|COOMe->COOH_',
    'max_rxn' : 1,
    'msg' : '* ester group(s) on methyl-benzoate hydrolysised into carboxylic acid. '
}

all_ms_Est2Acd = {
    'rxn_n' : [],
    'ms' : [],
    'msg' : [],
    'name' : []
}

# calculate all the possible combinations with follow-up ester hydrolysis

for i in range(Est2Acd['max_rxn'] + 1):

    for k, et_ms in enumerate(Est2Acd['st_ms_lst']):

        new_ms = et_ms + Est2Acd['updt_ms'] * i
        # print(new_ms)
        pre_msg = all_ms_Ol2Et['msg'][k]
        new_msg = pre_msg + str(i) + ' ' + Est2Acd['msg']
        # print(new_msg)
        rxn_type = Est2Acd['rxn']
        pre_rxn = all_ms_Ol2Et['name'][k]
        all_ms_updt(all_ms_Est2Acd, i, new_ms, new_msg, rxn_type, pre_rxn = pre_rxn)

all_ms_Est2Acd['ms_with_Na'] = [x + 23.990 - 1.008 for x in all_ms_Est2Acd['ms']]
pprint.pprint(all_ms_Est2Acd)

{'ms': [184.15,
        352.47400000000005,
        520.798,
        689.1220000000001,
        170.12300000000002,
        338.44700000000006,
        506.771,
        675.095],
 'ms_with_Na': [207.132,
                375.4560000000001,
                543.78,
                712.104,
                193.10500000000002,
                361.4290000000001,
                529.7529999999999,
                698.077],
 'msg': ['0 * OH group(s) on trioxyl methyl-benzoate substituted by dodecyl '
         'group(s). 0 * ester group(s) on methyl-benzoate hydrolysised into '
         'carboxylic acid. ',
         '1 * OH group(s) on trioxyl methyl-benzoate substituted by dodecyl '
         'group(s). 0 * ester group(s) on methyl-benzoate hydrolysised into '
         'carboxylic acid. ',
         '2 * OH group(s) on trioxyl methyl-benzoate substituted by dodecyl '
         'group(s). 0 * ester group(s) on methyl-benzoate hydrolysised into '
         'carboxylic acid. ',
         '3 * OH group

Step 3: Predict side reactions that may occur between suspicious solvents.

In [96]:
# potential side rxns triggered by solvents

solv_N_lp = ['NMI', 'Pyd', 'TEA'] # solvents with lone pairs on N can possibly form ionic liquid with BrAlkane or attach at the ester/acid site. ACN requires further check.
solv_OH = ['nPA', 'EtOH', 'IPA', 'tBuOH'] # solvents with OH groups can possibly form ether with BrAlkane via SN2 pathway and E2 product with the alcohol itself. It may also trigger transesterification at the ester/acid site.
solv_est = ['EA'] # solvents contain ester group may trigger transesterification at the ester/acid site.

solv_side_rxn = {
    'N_lp' : ['IL', 'EstAtk'],
    'OH' : ['SN2E2', 'EstAtk'],
    'est' : ['EstExg']
}

In [97]:
ms_BrAlk = 249.24

# open solvents with suspcious properties

f = open('susp_solv.json')
susp_solv_data = json.load(f)

pprint.pprint(susp_solv_data)

[{'full_name': 'N-methylimidazole',
  'ms': 82.1,
  'name': 'NMI',
  'susp_prop': 'N_lp'},
 {'full_name': 'Pyridine', 'ms': 79.1, 'name': 'Pyd', 'susp_prop': 'N_lp'},
 {'full_name': 'Triethylamine',
  'ms': 101.19,
  'name': 'TEA',
  'susp_prop': 'N_lp'},
 {'full_name': '1-propanol', 'ms': 60.09, 'name': 'nPA', 'susp_prop': 'OH'},
 {'full_name': 'ethanol', 'ms': 46.07, 'name': 'EtOH', 'susp_prop': 'OH'},
 {'full_name': 'isopropanol', 'ms': 60.1, 'name': 'IPA', 'susp_prop': 'OH'},
 {'full_name': 'tertbutylalcohol',
  'ms': 74.12,
  'name': 'tBuOH',
  'susp_prop': 'OH'},
 {'exg_frag_ms': 46.01,
  'full_name': 'ethyal_acetate',
  'ms': 88.11,
  'name': 'EA',
  'susp_prop': 'est'}]


In [98]:
side_prod_IL = {

    'side_rx_type' : 'IL',
    'ms' : [],
    'msg' : [],
    'name' : []

}


def side_rx_IL(susp_solv):

    prod_name = susp_solv['name'] + '_' + 'BrAlkane' + '_salt'
    prod_ms = susp_solv['ms'] + ms_BrAlk
    prod_msg = 'Ionic liuqid is formed between ' + susp_solv['full_name'] + ' and 1-bromododecane.'

    side_prod_IL['ms'].append(prod_ms)
    side_prod_IL['name'].append(prod_name)

    side_prod_IL['msg'].append(prod_msg)

    return side_prod_IL

In [99]:
side_prod_SN2 = {

    'side_rx_type' : 'SN2',
    'ms' : [],
    'msg' : [],
    'name' : []

}

def side_rx_SN2(susp_solv):
    prod_name = susp_solv['name'] + '_' + 'BrAlkane' + '_SN2'
    prod_ms = susp_solv['ms'] + ms_BrAlk - 80.912
    prod_msg = 'SN2 product is formed between ' + susp_solv['full_name'] + ' and 1-bromododecane.'

    side_prod_SN2['ms'].append(prod_ms)
    side_prod_SN2['name'].append(prod_name)

    side_prod_SN2['msg'].append(prod_msg)

    return side_prod_SN2

In [100]:
side_prod_E2 = {

    'side_rx_type' : 'E2',
    'ms' : [],
    'msg' : [],
    'name' : []

}

def side_rx_E2(susp_solv):
    prod_name = susp_solv['name'] + '_' + 'BrAlkane' + '_E2'
    prod_ms = ms_BrAlk - 80.912 # One BrAlkane removes a HBr.
    prod_msg = 'E2 product is formed from by 1-bromododecane when reacting with ' + susp_solv['full_name'] + '.'

    side_prod_E2['ms'].append(prod_ms)
    side_prod_E2['name'].append(prod_name)

    side_prod_E2['msg'].append(prod_msg)

    return side_prod_E2

In [101]:
side_prod_EstAtk = {

    'side_rx_type' : 'EstAtk',
    'ms' : [],
    'msg' : [],
    'name' : []

}

#ester will be extracted from all_ms_Ol2Et

def side_rx_EstAtk(susp_solv, ester):

    for i, est_ms in enumerate(ester['ms']):
        
        prod_ms = est_ms + susp_solv['ms'] - 32.042 # remove a methanol from benzoate
        prod_name = susp_solv['name'] + '_' + ester['name'][i] + '_new_ester'
        prod_msg = 'The methyl ester fraction on the ' + ester['name'][i] + ' is replaced by ' + susp_solv['full_name'] + ' .'

        side_prod_EstAtk['ms'].append(prod_ms)
        side_prod_EstAtk['msg'].append(prod_msg)
        side_prod_EstAtk['name'].append(prod_name)
    
    return side_prod_EstAtk


In [102]:
side_prod_EstExg = {

    'side_rx_type' : 'EstExg',
    'ms' : [],
    'msg' : [],
    'name' : []

}

#ester will be extracted from all_ms_Ol2Et

def side_rx_EstExg(susp_solv, ester):

    # if susp_solv['name'] == 'EA':
    #     susp_solv['exg_frag_ms'] = 46.07 # this is the equivalent mass of alcohol to be replaced
    
    try:

        for i, est_ms in enumerate(ester['ms']):
            
            prod_ms = est_ms + susp_solv['exg_frag_ms'] - 32.042 # remove a methanol from benzoate
            prod_name = susp_solv['name'] + '_' + ester['name'][i] + '_new_ester'
            prod_msg = 'The methyl ester fraction on the ' + ester['name'][i] + ' is replaced by the equivalent alcohol part of the ' + susp_solv['full_name'] + ' .'

            side_prod_EstExg['ms'].append(prod_ms)
            side_prod_EstExg['msg'].append(prod_msg)
            side_prod_EstExg['name'].append(prod_name)
    
    except susp_solv['exg_frag_ms'] == None:

        print('This is not an ester.')


    return side_prod_EstExg

Based on the suspicious properties of solvents (submitted from the susp_solv_json), activate different side rxn mechanism and calculations.

In [103]:
for a, susp_solv in enumerate(susp_solv_data):

    if susp_solv['susp_prop'] == 'N_lp':

        side_prod_IL = side_rx_IL(susp_solv)
        side_prod_EstAtk = side_rx_EstAtk(susp_solv, all_ms_Ol2Et)
    
    elif susp_solv['susp_prop'] == 'OH':

        side_prod_SN2 = side_rx_SN2(susp_solv)
        side_prod_E2 = side_rx_E2(susp_solv)
        side_prod_EstAtk = side_rx_EstAtk(susp_solv, all_ms_Ol2Et)
    
    elif susp_solv['susp_prop'] == 'est':

        side_prod_EstExg = side_rx_EstExg(susp_solv, all_ms_Ol2Et)

#print(side_prod_IL, side_prod_EstAtk, side_prod_SN2, side_prod_E2, side_prod_EstExg)

TD

For this specific reaction, it is also possible to have oligomers formed by the non-/partially subsituted starting materials themselves as well. The unreacted ester site will follow the same situation with hydrolysis or reacting with suspicious solvents.

The possible combination is between substitution degree from 0 to 2 (because 3 does not have free OH groups).

In [104]:
def calc_eff_ms_acd(ori_est_ms):

    ms_eff_acd = ori_est_ms - 16.043 # the ester end remove the methyl group and add a H to form the effective acid structrue before eliminating a water.

    return ms_eff_acd

def eff_acd_msg_updt(ori_est_msg):

    eff_acd_msg = 'Eq acid worked out from the ester: ' + ori_est_msg

    return eff_acd_msg

def est_plm_ms(ori_est_eff_acd_ms, sec_est_ms):

    ply_est_ms = ori_est_eff_acd_ms + sec_est_ms - 18.015 # remove a water

    return ply_est_ms


def ply_est_ms_lmt_chk(ply_est_ms):

    if ply_est_ms > 1000:

        ply_est_vld_msg = ['Mass exceeds the limit of ESI/Nanospray', 'False']
    
    else:

        ply_est_vld_msg = ['Mass valid', 'True']

    return ply_est_vld_msg


In [105]:

# generate the un-fully occupied ms of starting ester. Call them reactive species due to the free OH group.
all_ms_PlyEstReactive = {
    'rxn_n' : [],
    'ms' : [],
    'msg' : [],
    'name' : []
}


# The initial species with free ester site able to be attached with the analogous of themselves

all_ms_PlyEstInit = {
    'rxn_n' : [],
    'ms' : [],
    'msg' : [],
    'name' : []
}


for i in range(len(all_ms_Ol2Et['ms'])):

    crt_est_ms = all_ms_Ol2Et['ms'][i]
    crt_est_msg = all_ms_Ol2Et['msg'][i]
    crt_est_name = all_ms_Ol2Et['name'][i]
    crt_est_rxn_n = all_ms_Ol2Et['rxn_n'][i]

    ms_eff_acd_updt = calc_eff_ms_acd(crt_est_ms)
    msg_eff_acd_updt = eff_acd_msg_updt(crt_est_msg)

    all_ms_PlyEstInit['rxn_n'].append(crt_est_rxn_n)
    all_ms_PlyEstInit['ms'].append(ms_eff_acd_updt)
    all_ms_PlyEstInit['msg'].append(msg_eff_acd_updt)
    all_ms_PlyEstInit['name'].append(crt_est_name + '_eff_acd')

    if crt_est_rxn_n < 3:

        all_ms_PlyEstReactive['rxn_n'].append(crt_est_rxn_n)
        all_ms_PlyEstReactive['ms'].append(crt_est_ms)
        all_ms_PlyEstReactive['msg'].append(crt_est_msg)
        all_ms_PlyEstReactive['name'].append(crt_est_name)


pprint.pprint(all_ms_PlyEstReactive)
pprint.pprint(all_ms_PlyEstInit)




{'ms': [184.15, 352.47400000000005, 520.798],
 'msg': ['0 * OH group(s) on trioxyl methyl-benzoate substituted by dodecyl '
         'group(s). ',
         '1 * OH group(s) on trioxyl methyl-benzoate substituted by dodecyl '
         'group(s). ',
         '2 * OH group(s) on trioxyl methyl-benzoate substituted by dodecyl '
         'group(s). '],
 'name': ['smOH->OC12H25_0', 'smOH->OC12H25_1', 'smOH->OC12H25_2'],
 'rxn_n': [0, 1, 2]}
{'ms': [168.107, 336.43100000000004, 504.755, 673.0790000000001],
 'msg': ['Eq acid worked out from the ester: 0 * OH group(s) on trioxyl '
         'methyl-benzoate substituted by dodecyl group(s). ',
         'Eq acid worked out from the ester: 1 * OH group(s) on trioxyl '
         'methyl-benzoate substituted by dodecyl group(s). ',
         'Eq acid worked out from the ester: 2 * OH group(s) on trioxyl '
         'methyl-benzoate substituted by dodecyl group(s). ',
         'Eq acid worked out from the ester: 3 * OH group(s) on trioxyl '
         'met

In [106]:
PlyEstStack = {

    'ply_rxn_n' : [],
    'ms' : [],
    'msg' : [],
    'name' : [],
    'dop' : 1,
    'sum_rxn_n' : []
    
}

# calculate the combined mass after transesterification between the effective acid in the initial species and the reactive species with open OH slots.

init_ply_est = len(PlyEstStack['ms'])

calc_done = False

for i, init_eff_ms in enumerate(all_ms_PlyEstInit['ms']):    

    for j, reactive_est_ms in enumerate(all_ms_PlyEstReactive['ms']):

        comb_est_ms = est_plm_ms(init_eff_ms, reactive_est_ms)

        ply_est_vld_msg = ply_est_ms_lmt_chk(comb_est_ms)

        # print(ply_est_vld_msg)
        ply_est_name = [all_ms_PlyEstInit['name'][i], all_ms_PlyEstReactive['name'][j]]

        if ply_est_vld_msg[-1] == 'True':

            PlyEstStack['ms'].append(comb_est_ms)
            PlyEstStack['ply_rxn_n'].append([all_ms_PlyEstInit['rxn_n'][i], all_ms_PlyEstReactive['rxn_n'][j]])

            PlyEstStack['sum_rxn_n'].append(np.sum(PlyEstStack['ply_rxn_n'][-1]))

            PlyEstStack['name'].append(ply_est_name)
            PlyEstStack['msg'].append('Polyester formed from: ' + str(ply_est_name))

        
        else:

            ivld_msg = '[MassTooLargeError] Polyester formed from: ' + str(ply_est_name) + ' ' + str(ply_est_vld_msg[0]) + '. The calculated ms is: ' + str(comb_est_ms)

            print(ivld_msg)
        
vld_ply_est = len(PlyEstStack['ms'])



if vld_ply_est - init_ply_est == 0:

    calc_done = True

    print('No more new species.')

else:

    PlyEstStack['dop'] += 1


pprint.pprint(PlyEstStack)



[MassTooLargeError] Polyester formed from: ['smOH->OC12H25_2_eff_acd', 'smOH->OC12H25_2'] Mass exceeds the limit of ESI/Nanospray. The calculated ms is: 1007.5379999999999
[MassTooLargeError] Polyester formed from: ['smOH->OC12H25_3_eff_acd', 'smOH->OC12H25_1'] Mass exceeds the limit of ESI/Nanospray. The calculated ms is: 1007.5380000000001
[MassTooLargeError] Polyester formed from: ['smOH->OC12H25_3_eff_acd', 'smOH->OC12H25_2'] Mass exceeds the limit of ESI/Nanospray. The calculated ms is: 1175.8619999999999
{'dop': 2,
 'ms': [334.242,
        502.56600000000003,
        670.89,
        502.56600000000003,
        670.8900000000001,
        839.214,
        670.89,
        839.214,
        839.214],
 'msg': ["Polyester formed from: ['smOH->OC12H25_0_eff_acd', "
         "'smOH->OC12H25_0']",
         "Polyester formed from: ['smOH->OC12H25_0_eff_acd', "
         "'smOH->OC12H25_1']",
         "Polyester formed from: ['smOH->OC12H25_0_eff_acd', "
         "'smOH->OC12H25_2']",
       

When we go beyond dimers (i.e., degree of polymerisation > 2), we need to play around the "degree of freedom" indicated by the available reaction site.

Let's do some quick math:

DoP reflects how many reactions have occurred already.

For each reaction, 1 DoF needs to be removed.

set n = DoP for simplicity,

Total number of molecules participated in the reaction would be (n+1).

Total OH groups (susppose no substitution) is (n+1)*3.

The polymerisation is only possible when the total substitution number in the system does not exceed:

(n+1)*3-n

= 2n + 3

A reaction would otherwise be forbidden if the addition of the next molecule exceed this number. (Note that here the "n" is the DoP before adding the next molecule.)


Here, since we only consider the mass, we ignore the possible isomers for simplicity.

In [107]:
vld_ply_est = len(PlyEstStack['ms'])

reactive_stack = all_ms_PlyEstReactive
reactive_stack['sum_rxn_n'] = all_ms_PlyEstReactive['rxn_n']

init_stack = all_ms_PlyEstInit

def form_rxn_n(ori_rxn_n):

    new_rxn_n = []

    for x in ori_rxn_n:

        new_rxn_n.append([x])
    
    return new_rxn_n

new_rxn_n_rxt_stk = form_rxn_n(reactive_stack['rxn_n'])

new_rxn_n_init_stk = form_rxn_n(init_stack['rxn_n'])

reactive_stack['rxn_n'] = new_rxn_n_rxt_stk
init_stack['rxn_n'] = new_rxn_n_init_stk

print('reactive_stack: \n')
pprint.pprint(reactive_stack)

print('init_stack: \n')
pprint.pprint(init_stack)

reactive_stack: 

{'ms': [184.15, 352.47400000000005, 520.798],
 'msg': ['0 * OH group(s) on trioxyl methyl-benzoate substituted by dodecyl '
         'group(s). ',
         '1 * OH group(s) on trioxyl methyl-benzoate substituted by dodecyl '
         'group(s). ',
         '2 * OH group(s) on trioxyl methyl-benzoate substituted by dodecyl '
         'group(s). '],
 'name': ['smOH->OC12H25_0', 'smOH->OC12H25_1', 'smOH->OC12H25_2'],
 'rxn_n': [[0], [1], [2]],
 'sum_rxn_n': [0, 1, 2]}
init_stack: 

{'ms': [168.107, 336.43100000000004, 504.755, 673.0790000000001],
 'msg': ['Eq acid worked out from the ester: 0 * OH group(s) on trioxyl '
         'methyl-benzoate substituted by dodecyl group(s). ',
         'Eq acid worked out from the ester: 1 * OH group(s) on trioxyl '
         'methyl-benzoate substituted by dodecyl group(s). ',
         'Eq acid worked out from the ester: 2 * OH group(s) on trioxyl '
         'methyl-benzoate substituted by dodecyl group(s). ',
         'Eq acid worked

In [108]:
def vld_nxt_rxn(crt_dop, crt_rxn_n, nxt_rxn_n):

    nxt_rxn_vld = True

    total_rxn_n = crt_rxn_n + nxt_rxn_n

    rxn_n_lmt = 2*crt_dop + 3

    if total_rxn_n > rxn_n_lmt:

        nxt_rxn_vld = False
    
    return nxt_rxn_vld



def calc_done(vld_ply_est, init_ply_est):

    calc_done = False

    if vld_ply_est - init_ply_est == 0:

        calc_done = True

        print('Calculation done. \n')
    
    return calc_done

In [ ]:
for a, a_ms in enumerate(PlyEstStack['ms']):

    sub_crt = PlyEstStack['sum_rxn_n'][a]
    crt_dop = PlyEstStack['dop']

    for b, b_ms in enumerate(reactive_stack['ms']):

        sub_next = reactive_stack('sum_rxn_n')[b]

        nxt_rxn_vld = vld_nxt_rxn(crt_dop, sub_crt, sub_next)

        if nxt_rxn_vld is True:

            ms_eff_acd = calc_eff_ms_acd(a_ms)
            msg_eff_acd = eff_acd_msg_updt(PlyEstStack['name'][a])
            init_stack['ms'].append(ms_eff_acd)
            init_stack['msg'].append(msg_eff_acd)
            init_stack['name'].append(PlyEstStack['name'][a] + '|_eff_acd')
            init_stack['rxn_n'].append(PlyEstStack['ply_rxn_n'])

            

Output

In [109]:
from datetime import datetime
now = datetime.now()

dt_string = now.strftime("%m%d%Y%H%M%S")
print(dt_string)

05082023131411


In [110]:
def decent_output_solv_side(side_rxn_log_dict):
    side_rxn_type = str(list(side_rxn_log_dict.keys())[0]) + ' : ' + side_rxn_log_dict['side_rx_type'] + '\n'
    f.write('========================== \n')
    f.write(side_rxn_type)

    len_val = len(list(side_rxn_log_dict['ms']))

    for i in range(len_val):
        grp_id = 'Group ' + str(i + 1) + ': \n'
        f.write('\n ------ \n')
        f.write(grp_id)
        f.write('\n')

        for k, key in enumerate(list(side_rxn_log_dict.keys())):

            if k != 0:        
            
                cont = str(key) + ' : ' + str(side_rxn_log_dict[key][i]) + '\n'
                f.write(cont)
    
    f.write('\n =========================== \n')


# def decent_output_std_subst_prod(std_subst_dict):
#     f.write('Standard substitution products to be expected: \n\n')

#     for k, key in enumerate(list(std_subst_dict.keys())):

#         for i, item in enumerate(list(std_subst_dict[key])):
            
#             grp_id = 'Group ' + str([k, i]) + ': \n'
#             cont = str(key) + ' : ' + str(item) + '\n\n'
#             f.write(grp_id)
#             f.write(cont)
    
#     f.write('=========================== \n')

def decent_output_std_hydrl_prod(std_hydrl_dict):
    # f.write('Hydrolysis products to be expected: \n\n')
    len_val = len(list(std_hydrl_dict['ms']))

    for i in range(len_val):
        grp_id = 'Group ' + str(i + 1) + ': \n'
        f.write('\n ------ \n')
        f.write(grp_id)

        for k, key in enumerate(list(std_hydrl_dict.keys())):        
            
            cont = str(key) + ' : ' + str(std_hydrl_dict[key][i]) + '\n'
            f.write(cont)
    
    f.write('\n =========================== \n')

    # for k, key in enumerate(list(std_hydrl_dict.keys())):

    #     grp_id = 'Group ' + str(k) + ': \n'
    #     cont = str(key) + ' : ' + str(std_hydrl_dict[key]) + '\n\n'
    #     f.write(grp_id)
    #     f.write(cont)
    
    # f.write('=========================== \n')


In [111]:
log_name = 'rxn_anls_log_' + str(dt_string) + '.txt'

with open(log_name, 'w') as f:
    
    f.write(str(now) + '\n')
    # decent_output_std_subst_prod(all_ms_Ol2Et)
    f.write('==================== \n')
    f.write('Potential side products not induced by solvents: \n')
    f.write('==================== \n')
    
    decent_output_std_hydrl_prod(all_ms_Est2Acd)

    # f.write('==================== \n')
    f.write('Potential side products induced by solvents: \n\n')

    all_solv_side = [side_prod_IL, side_prod_EstAtk, side_prod_SN2, side_prod_E2, side_prod_EstExg]

    for s, rxn in enumerate(all_solv_side):
        
        rxn['ms_with_Na'] = [x + 23.990 - 1.008 for x in rxn['ms']]

        decent_output_solv_side(rxn)
    
print('Please check: ' + str(log_name) + ' for output.')

Please check: rxn_anls_log_05082023131411.txt for output.


Below are irrelevant tests...

In [112]:
atom_mass_dict = {
    'C' : 12.011,
    'H' : 1.008,
    'O' : 15.999,
    'N' : 14.007,
    'S' : 32.06,
    'Br' : 79.904
}



In [113]:
rx_comb = {
    'CBr' : [['ACOH', [['H', -1], ['Br', -1]]], ['COH', [['H', -1], ['Br', -1]]], ['ACN', []]]
    
}

ind_solvent_from_db_DMF= {
    'name' : 'DMF',
    'formula' : [['C', 3], ['H', 7], ['N', 1], ['O', 1]],
    'basicity' : 0,
    'rx_site' : None
}

ind_solvent_from_db_NMI= {
    'name' : 'DMF',
    'formula' : [['C', 3], ['H', 7], ['N', 1], ['O', 1]],
    'kamlet_taft' : [0, 0.69, 0.88],
    'basicity' : 0,
    'rx_site' : [['ACN', 1]]
}

ind_sm_acoh_est = {
    'name' : 'ACOH_est',
    'formula' : [['C', 8], ['H', 8], ['O', 5]],
    'rx_site' : [['ACOH', 3], ['COOR', 1]]
}

ind_sm_br_alk = {
    'name' : 'Br_alk',
    'formula' : [['C', 12], ['H', 25], ['Br', 1]],
    'rx_site' : [['CBr', 1]]
}

In [114]:
all_sm = ['ACOH_est', 'Br_alk']
all_solv = ['DMF']